First we instantiate the simulator class we will use to generate our sequences. The Simulator handles all the particulars about the target being tracked. This simulation in particular is a Linear Time Invariant scenario of an object travelling at a constant velocity in two dimensions. However, it is being observed via a nonlinear measurement model.

In particular it is a Bearing Range sensor positioned at the origin of the coordinate system. Measurements are collected as a 2 dimensional vector containing an angle, and a distance.

In [1]:
from stonesoup.types.groundtruth import GroundTruthPath, GroundTruthState
from stonesoup.types.detection import Detection

from stonesoup.types.track import Track
from stonesoup.types.hypothesis import SingleHypothesis

class simulator():

    def __init__(self, transition_model, measurement_model, state_range, meas_range = None, noise_range = None, meas_noise_range = None):
        """
        Parameters
        ----------
        transition_model : :class:`~.Predictor`
            The Stone Soup predictor to be used.
        measurement_model : :class:`~.Predictor`
            The Updater to be used.
        """
        self.transition_model= transition_model
        self.measurement_model = measurement_model
        self.state_range = state_range
        self.meas_range = meas_range
        self.noise_range = noise_range
        self.meas_noise_range = meas_noise_range

    def _generate_ground_truth(self, prior, time_span):
        
        time_interval = time_span[1]-time_span[0]
        ground_truth = GroundTruthPath([prior])
        for k in range(1, len(time_span)):
            ground_truth.append(GroundTruthState(
                self.transition_model.function(ground_truth[k-1], noise=True, time_interval=time_interval),
                timestamp=time_span[k-1]))
        return ground_truth
    
    def _simulate_measurements(self, ground_truth):
        #Simulate Measurements
        measurements = []
        for state in ground_truth:
            measurement = self.measurement_model.function(state, noise=True)
            measurements.append(Detection(measurement,
                                          timestamp=state.timestamp,
                                          measurement_model=self.measurement_model))
        return measurements

    def _initializer(self, time_stamp):
        state_range_min = self.state_range[0]
        state_range_max = self.state_range[1]
        state_vector = StateVector(np.random.uniform(low=state_range_min, high=state_range_max))
        while np.sqrt(state_vector[1]**2+state_vector[3]**2) < 0.15:
            state_vector = StateVector(np.random.uniform(low=state_range_min, high=state_range_max))
        meas_range_low = self.meas_range[0]
        meas_range_max = self.meas_range[1]
        self.measurement_model.translation_offset = np.random.uniform(low=meas_range_low, high=meas_range_max).reshape(len(meas_range_low),1)
        #process_noise = 
        #self.transition_model. = np.randint(low=0,high=len(noise_range))
        
        return State(state_vector = state_vector, timestamp=time_stamp)

    def generate_training_data(self, time_span):
        """

        Parameters
        ----------
        ground_truth : :class:`~.GroundTruthPath`
            StateMutableSequence type object used to store ground truth.
        initial_state : :class:`~.State`
            Initial state for the ground truth system. This MUST be a State,
            not a State subclass, like GaussianState or EnsembleState.
        prior : :class:`~.GaussianState` or :class:`~.EnsembleState`
            Initial state prediction of tracking algorithm.

        Returns
        -------
        track : :class:`~.Track`
            The Stone Soup track object which contains the list of updated 
            state predictions made by the tracking algorithm employed.
        """

        #Simulate Measurements
        initial_state = self._initializer(time_span[0])
        ground_truth = self._generate_ground_truth(initial_state, time_span)
        measurements = self._simulate_measurements(ground_truth)
        
        return ground_truth, measurements

    def simulate_track(self, predictor, updater, initial_state, prior, time_span):
        """

        Parameters
        ----------
        predictor : :class:`~.Predictor`
            The Stone Soup predictor to be used.
        updater : :class:`~.Predictor`
            The Updater to be used.
        ground_truth : :class:`~.GroundTruthPath`
            StateMutableSequence type object used to store ground truth.
        initial_state : :class:`~.State`
            Initial state for the ground truth system. This MUST be a State,
            not a State subclass, like GaussianState or EnsembleState.
        prior : :class:`~.GaussianState` or :class:`~.EnsembleState`
            Initial state prediction of tracking algorithm.

        Returns
        -------
        track : :class:`~.Track`
            The Stone Soup track object which contains the list of updated 
            state predictions made by the tracking algorithm employed.
        """

        #Simulate Measurements
        ground_truth = self._generate_ground_truth(initial_state, time_span)
        measurements = self._simulate_measurements(ground_truth)
        
        #Initialize Loop Variables
        track = Track()
        for measurement in measurements:
            prediction = predictor.predict(prior, timestamp=measurement.timestamp)
            hypothesis = SingleHypothesis(prediction, measurement)  # Group a prediction and measurement
            posterior = updater.update(hypothesis)
            track.append(posterior)
            prior = track[-1]
        return ground_truth, track

In [2]:
import numpy as np
import matplotlib.pyplot as plt
import datetime

from stonesoup.types.array import StateVector
from stonesoup.types.state import State, GaussianState

from stonesoup.models.transition.linear import (CombinedLinearGaussianTransitionModel,
                                                ConstantVelocity)
from stonesoup.models.measurement.nonlinear import CartesianToBearingRange

"""
    This script represents the code used to gather the data used in [PAPER HERE].
    
    This repository is structured such that different stone soup algorithms 
    can be run rapidly. Hopefully I've made it modular enough to 
    allow swapping of things like algorithms, and "experiments" by replacing
    the desired transition and measurement models.
    
    The simulator class requires a transition and 
    measurement model, then the simulate_track method accepts a Stone Soup
    Predictor, Updater, ground truth initial state, initial state for the
    chosen algorithm, and a span of time which the simulation takes place over.
    This time span should be an evenly spaced datetime.datetime list.
    
    The simulator then, is used to gather "Track" instances, and with a list 
    of tracks, RMSE can then be calculated.
"""

i = 60
num_vectors = i*5

"""
    Here, we get our initial variables for simulation. For this, we are just
    using a time span of 60 time instances spaced one second apart.
"""

timestamp = datetime.datetime(2021, 4, 2, 12, 28, 57)
tMax = 60
dt = 1
tRange = tMax // dt
plot_time_span = np.array([dt*i for i in range(tRange)])

time_span = np.array([timestamp + datetime.timedelta(seconds=dt*i) for i in range(tRange)])

"""
Here we instantiate our transition and measurement models. These are the 
same models used in the StoneSoup Kalman Filter examples.
"""

q_x = 0.05
q_y = 0.05
sensor_x = 0  # Placing the sensor off-centre
sensor_y = 0

transition_model = CombinedLinearGaussianTransitionModel([ConstantVelocity(q_x),
                                                          ConstantVelocity(q_y)])

measurement_model = CartesianToBearingRange(
ndim_state=4,
mapping=(0, 2),
noise_covar=np.diag([np.radians(1)**2, 0.1**2]),  # Covariance matrix. 0.2 degree variance in
# bearing and 1 metre in range
translation_offset=np.array([[sensor_x], [sensor_y]])  # Offset measurements to location of
# sensor in cartesian.
)


In order to use this simulated data, we need to format it such that it can be loaded into PyTorch easily. Pytorch requires us to specify functions "__get_item__" and "__len__" in order to be usable by the generic Data Loader. I simply included the init and a helper function to append trajectories to the dataset for ease of use.

In [3]:
import torch
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
from torch.utils.data import Dataset

import numpy as np

# Datasets should be specified on a per simulation basis

#class sequence_generator(Dataset):

class dataset_2D_bearing_range(Dataset):
    #Range Bearing 2D Dataset Sequence Packer

    def __init__(self, dataset, ground_truth=None, measurements=None):
        if dataset == None:
            gt = np.array([e.state_vector for e in ground_truth]).squeeze().T
            gt = torch.tensor(gt.astype(dtype=np.float32),device = device)
            ms = np.array([m.state_vector for m in measurements]).squeeze().T
            ms = torch.tensor(ms.astype(dtype=np.float32),device = device)
            self.dataset = torch.unsqueeze(torch.cat((ms,gt)),dim=0)
        else:
            self.dataset = dataset


    def append_to_dataset(self, ground_truth, measurements):
        """
        Parameters
        ----------
        ground_truth : :class:`~.list`
            A list of Stonesoup States.
        measurements : :class:`~.measurements`
            The list of Stonesoup Measurements to be used.
        """

        gt = np.array([e.state_vector for e in ground_truth]).squeeze().T
        gt = torch.tensor(gt.astype(dtype=np.float32),device = device)
        ms = np.array([m.state_vector for m in measurements]).squeeze().T
        ms = torch.tensor(ms.astype(dtype=np.float32),device = device)
        new_entry = torch.cat((ms,gt),dim=0)

        self.dataset = torch.cat((self.dataset,torch.unsqueeze(new_entry,dim=0)),dim=0)

    def __getitem__(self, idx):
        return self.dataset[idx,0:2].T, self.dataset[idx,2:6].T

    def __len__(self):
        return self.dataset.shape[0]


This next module is something I call the Composer. The Composer's job is to return a useful dataset for use in PyTorch, and I will tell it how many trajectories I want, and it will call the relevant simulator and dataset methods to generate a dataset for the purpose of learning.

In [4]:
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader

import numpy as np

# Datasets should be specified on a per simulation basis
# Should look up realistic values for radar precision and initial states
# Come up with realistic constraints for initial velocities

#class sequence_generator(Dataset):

class dataset_composer():
    #Range Bearing 2D Dataset Sequence Packer
    def __init__(self, simulators, training_dataset, testing_dataset, batch_size, num_workers):
        #pass a list of simulators
        self.simulators = simulators
        self.training_dataset = training_dataset
        self.testing_dataset = testing_dataset
        self.batch_size = batch_size
        self.num_workers = num_workers

    def simulate_trajectories(self, training_dataset_len, testing_ratio):
        for simulator in self.simulators:
            trainset_length = int((training_dataset_len-1) / len(self.simulators)) # Partition dataset into sets from each simulator
            for i in range(trainset_length):
                ground_truth, measurements = simulator.generate_training_data(time_span)
                self.training_dataset.append_to_dataset(ground_truth, measurements)
            trainloader = DataLoader(self.training_dataset)

            testset_length = int(((training_dataset_len)*testing_ratio -1)/len(self.simulators))
            for i in range(testset_length):
                ground_truth, measurements = simulator.generate_training_data(time_span)
                self.testing_dataset.append_to_dataset(ground_truth, measurements)
            testloader = DataLoader(self.testing_dataset)
        return trainloader, testloader

    def output_to_file(self, trainset_name, testset_name):
        torch.save(self.training_dataset, trainset_name)
        torch.save(self.testing_dataset, testset_name)

    def read_from_file(self, training_file_name, testing_file_name, device):
        trn = torch.load(training_file_name, map_location = device)
        tst = torch.load(testing_file_name, map_location = device)
        return DataLoader(dataset = trn, batch_size = self.batch_size, 
                          num_workers=self.num_workers, shuffle=True), DataLoader(dataset = tst, batch_size = self.batch_size,
                                                                                  num_workers=self.num_workers, shuffle = True)




Here we instantiate the simulator, provide a range of permissable values for the initial conditions, noise coefficients, and specify the transition model, and measurement model.



In [5]:
# Position, Velocity, Position, Velocity
state_range_min = np.array([-15, -2, -15, -2])
state_range_max = np.array([15, 2, 15, 2])

# Add some sort of miniumum speed

meas_range_min = np.array([0, 0])
meas_range_max = np.array([0, 0])

# Randomization for process noise
#process_noise_coefficients = [0.005, 0.05, 0.5]
process_noise_coefficients = [0.05]

nonlinear_simulator = simulator(transition_model=transition_model,
                      measurement_model=measurement_model,
                      state_range = (state_range_min, state_range_max),
                      meas_range = (meas_range_min, meas_range_max)
)

ground_truth, measurements = nonlinear_simulator.generate_training_data(time_span)
training_dataset =  dataset_2D_bearing_range(dataset = None, ground_truth = ground_truth, measurements = measurements)
testing_dataset =  dataset_2D_bearing_range(dataset = None, ground_truth = ground_truth, measurements = measurements)

#training_dataset_len = 1000000 #  1,000,000
training_dataset_len = 100000 #  100,000
#training_dataset_len = 10000
testing_ratio = 0.15

#batch_size = int(2**9) # 512, ?s per epoch
#batch_size = int(2**8) # 256, 38s per epoch
#batch_size = int(2**7) # 128, 38s per epoch
#batch_size = int(2**6) # 64, 72s per epoch
#batch_size = int(2**5) # 32, 138s per epoch
batch_size = int(2**9)
num_workers = 0

dataset_composer = dataset_composer([nonlinear_simulator], testing_dataset, training_dataset, batch_size = batch_size, num_workers = num_workers)

#trainloader, testloader = dataset_composer.simulate_trajectories(training_dataset_len, testing_ratio)
#dataset_composer.output_to_file('training_data','testing_data')
trainloader, testloader = dataset_composer.read_from_file('training_data', 'testing_data', device)
#print('Training Dataset Length:', training_dataset.__len__())
#print('Testing Dataset Length:',testing_dataset.__len__())


Turn Sequences into dataset

In [6]:
import torch.nn as nn

class UniDirectionalRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, nonlinearity, num_layers, device):
        super(UniDirectionalRNN, self).__init__()
        self.RNN = torch.nn.RNN(input_size, hidden_size, bidirectional=False, num_layers=num_layers, device=device)
        self.linear = torch.nn.Linear(hidden_size, output_size, device=device)

    def forward(self, input, RNN_hidden):
        RNN_output, RNN_hidden = self.RNN(input, RNN_hidden)
        output = self.linear(RNN_output)
        return output, RNN_hidden

class BiDirectionalRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, nonlinearity, num_layers, device):
        super(BiDirectionalRNN, self).__init__()
        self.RNN = torch.nn.RNN(input_size, hidden_size, bidirectional=True, num_layers=num_layers, device=device)
        self.linear = torch.nn.Linear(2*hidden_size, output_size, device=device)

    def forward(self, input, RNN_hidden):
        RNN_output, RNN_hidden = self.RNN(input, RNN_hidden)
        output = self.linear(RNN_output)
        return output, RNN_hidden

class BiDirectionalLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers, device):
        super(BiDirectionalLSTM, self).__init__()
        self.LSTM = torch.nn.LSTM(input_size, hidden_size, bidirectional=True, num_layers=num_layers, device=device)
        self.linear = torch.nn.Linear(2*hidden_size, output_size, device=device)

    def forward(self, input, LSTM_hidden, LSTM_cell):
        LSTM_output, (LSTM_hidden, LSTM_cell) = self.LSTM(input, (LSTM_hidden, LSTM_cell))
        output = self.linear(LSTM_output)
        return output, (LSTM_hidden, LSTM_cell)

class UniDirectionalLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers, device):
        super(UniDirectionalLSTM, self).__init__()
        self.LSTM = torch.nn.LSTM(input_size, hidden_size, bidirectional=False, num_layers=num_layers, device=device)
        self.linear = torch.nn.Linear(hidden_size, output_size, device=device)

    def forward(self, input, LSTM_hidden, LSTM_cell):
        LSTM_output, (LSTM_hidden, LSTM_cell) = self.LSTM(input, (LSTM_hidden, LSTM_cell))
        output = self.linear(LSTM_output)
        return output, (LSTM_hidden, LSTM_cell)

class EncoderDecoderLSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers, device):
        super(EncoderDecoderLSTM, self).__init__()
        self.encoder = torch.nn.LSTM(input_size, hidden_size, bidirectional=True, num_layers=num_layers, device=device)
        self.decoder = torch.nn.LSTM(hidden_size, hidden_size, bidirectional=True, num_layers=num_layers, device=device)
        self.linear = torch.nn.Linear(2*hidden_size, output_size, device=device)

    def forward(self, input, LSTM_hidden, LSTM_cell):
        encoder_output, (encoder_hidden, encoder_cell) = self.encoder(input, (LSTM_hidden, LSTM_cell))
        decoder_output, (decoder_hidden, decoder_cell) = self.decoder(input, (encoder_hidden, encoder_cell))
        output = self.linear(LSTM_output)
        return output, (encoder_hidden, encoder_cell), (decoder_hidden, decoder_cell)

class MHATransformer(nn.Module):
    def __init__(self, input_size=2, hidden_size=16, output_size = 4, nhead=4, num_encoder_layers=6, num_decoder_layers=6, dim_feedforward=2048, device=None):
        super(MHATransformer, self).__init__()
        self.input_layer = torch.nn.Linear(input_size, hidden_size, device=device)
        self.transformer = torch.nn.Transformer(d_model=hidden_size, nhead=nhead,
                                                num_encoder_layers=num_encoder_layers,
                                                num_decoder_layers=num_decoder_layers,
                                                dim_feedforward=dim_feedforward,
                                                batch_first=True,
                                                device=device)
        self.output_layer = torch.nn.Linear(hidden_size, output_size, device=device)

    def forward(self, input):
        hidden_input = self.input_layer(input)
        transformer_output = self.transformer(hidden_input, hidden_input)
        return self.output_layer(transformer_output)


Oogabooga

In [7]:
input_size = 2
output_size = 4

UniRNN_hidden_size = 128
UniRNN_num_layers = 3
nonlinearity = 'relu'

UniRNN = UniDirectionalRNN(input_size=input_size, hidden_size=UniRNN_hidden_size, 
                           output_size=output_size, nonlinearity=nonlinearity,
                           num_layers=UniRNN_num_layers, device=device)

BiRNN_hidden_size = 128
BiRNN_num_layers = 3
nonlinearity = 'relu'

BiRNN = BiDirectionalRNN(input_size=input_size, hidden_size=BiRNN_hidden_size, 
                         output_size=output_size, nonlinearity=nonlinearity, 
                         num_layers=BiRNN_num_layers, device=device)

UniLSTM_hidden_size = 128
UniLSTM_num_layers = 3
nonlinearity = 'relu'

UniLSTM = UniDirectionalLSTM(input_size=input_size, hidden_size=UniLSTM_hidden_size, 
                             output_size=output_size, num_layers=UniLSTM_num_layers,
                             device=device)

BiLSTM_hidden_size = 128
BiLSTM_num_layers = 3
nonlinearity = 'relu'

BiLSTM = BiDirectionalLSTM(input_size=input_size, hidden_size=BiLSTM_hidden_size, 
                           output_size=output_size, num_layers=BiLSTM_num_layers,
                           device=device)

print(BiLSTM.parameters())

transformer_hidden_size = 32
nhead = 16
num_encoder_layers = 6
num_decoder_layers = 6
dim_feedforward = 2048

Transformer = MHATransformer(input_size=input_size, hidden_size = transformer_hidden_size, 
                             output_size = output_size, nhead=nhead,
                             num_encoder_layers=num_encoder_layers, 
                             num_decoder_layers=num_decoder_layers, 
                             dim_feedforward=dim_feedforward, device=device)

<generator object Module.parameters at 0x000001A089351F20>


Initialize Optimizers

In [8]:
criterion = nn.MSELoss()
#UniRNN_learning_rate = 0.000005;
UniRNN_learning_rate = 0.00005;
UniRNN_weight_decay = 0.000 #L2 Regularization Factor
UniRNN_optimizer = torch.optim.Adam(UniRNN.parameters(),lr=UniRNN_learning_rate, weight_decay = UniRNN_weight_decay)
#UniRNN_scheduler = torch.optim.lr_scheduler.LinearLR(UniRNN_optimizer, start_factor=1,end_factor = 0.01, total_iters=50)

#BiRNN_learning_rate = 0.0000005;
BiRNN_learning_rate = 0.00005;
BiRNN_weight_decay = 0.000 #L2 Regularization Factor
BiRNN_optimizer = torch.optim.Adam(BiRNN.parameters(),lr=BiRNN_learning_rate, weight_decay = BiRNN_learning_rate)
#BiRNN_scheduler = torch.optim.lr_scheduler.LinearLR(BiRNN_optimizer, start_factor=1,end_factor = 0.01, total_iters=50)

#UniLSTM_learning_rate = 0.000005;
UniLSTM_learning_rate = 0.00005;
UniLSTM_weight_decay = 0.000 #L2 Regularization Factor
UniLSTM_optimizer = torch.optim.Adam(UniLSTM.parameters(),lr=UniLSTM_learning_rate, weight_decay = UniLSTM_weight_decay)
#UniLSTM_scheduler = torch.optim.lr_scheduler.LinearLR(UniLSTM_optimizer, start_factor=1,end_factor = 0.01, total_iters=50)

#BiLSTM_learning_rate = 0.0000005;
BiLSTM_learning_rate = 0.00005;
BiLSTM_weight_decay = 0.000 #L2 Regularization Factor
BiLSTM_optimizer = torch.optim.Adam(BiLSTM.parameters(),lr=BiLSTM_learning_rate, weight_decay = BiLSTM_weight_decay)
#BiLSTM_scheduler = torch.optim.lr_scheduler.LinearLR(BiLSTM_optimizer, start_factor=1,end_factor = 0.01, total_iters=50)

#Transformer_Learning_rate = 0.0000005; # 0.0005 is is too fast, will not learn
Transformer_Learning_rate = 0.00005; # 0.0005 is is too fast, will not learn
TransformerWeight_decay = 0.000 #L2 Regularization Factor
Transformer_optimizer = torch.optim.Adam(Transformer.parameters(),lr=Transformer_Learning_rate, weight_decay = TransformerWeight_decay)
#Transformer_scheduler = torch.optim.lr_scheduler.LinearLR(Transformer_optimizer, start_factor=1, end_factor = 0.1, total_iters=50)

Lets run the EKF on the dataset to get a performance baseline. This is the class simulator to generate the tracks for the dataset.

In [9]:
class track_simulator():
    def __init__(self, predictor, updater, dataloader, time_span):
        """
        Parameters
        ----------
        transition_model : :class:`~.Predictor`
            The Stone Soup predictor to be used.
        measurement_model : :class:`~.Predictor`
            The Updater to be used.
        """
        self.predictor = predictor
        self.updater = updater
        self.dataloader = dataloader
        self.time_span = time_span

    def initializer(self, time_stamp, labels):
        state_range_min = labels[0][0].cpu().numpy()
        state_range_max = labels[0][1].cpu().numpy()
        state_vector = StateVector(np.random.uniform(low=state_range_min, high=state_range_max))
        return GaussianState(state_vector = state_vector, covar = np.eye(4), timestamp=time_stamp)

    def _simulate_track(self, inputs, labels):
        track = Track()
        prior = self.initializer(self.time_span[0], labels)
        for i in range(inputs.shape[1]):
            measurement = Detection(state_vector=inputs[0,i].cpu().numpy(), timestamp = self.time_span[i])
            prediction = self.predictor.predict(prior, timestamp=measurement.timestamp)
            posterior = self.updater.update(SingleHypothesis(prediction, measurement))
            track.append(posterior)
            prior = track[-1]
        tk = np.array([e.state_vector for e in track]).squeeze().T
        tk = torch.tensor(tk.astype(dtype=np.float32),device = device)
        return tk.T

    def generate_MSE(self):
        avg_loss = nn.MSELoss()
        trajectory_MSE = nn.MSELoss(reduction='none')
        MSE = torch.zeros(1,tMax,4, device=device) # Make this auto detect it
        test_loss = 0
        for j, data in enumerate(self.dataloader):
            inputs, labels = data
            outputs = self._simulate_track(inputs, labels)
            test_loss += avg_loss(outputs, labels.squeeze())
            MSE += trajectory_MSE(outputs, labels.squeeze())
        return test_loss / (j+1), MSE.squeeze()/len(self.dataloader)

Here we instantiate a testloader for use with the EKF (the EKF cannot handle batched inputs).

In [10]:
tst = torch.load('testing_data', map_location = device)
trajectory_testloader = DataLoader(tst)

In [11]:
from stonesoup.predictor.kalman import ExtendedKalmanPredictor
#from stonesoup.updater.kalman import ExtendedKalmanUpdater

#predictor = ExtendedKalmanPredictor(transition_model)
#updater = ExtendedKalmanUpdater(measurement_model)

#track_simulator = track_simulator(predictor, updater, trajectory_testloader, time_span)

#EKF_epoch_loss, EKF_track = track_simulator.generate_MSE()

#epoch_mse, EKF_MSE = track_simulator.generate_MSE()

#print(epoch_mse)
#plotter.two_by_two_MSE_vs_time([EKF_MSE], ['Extended Kalman Filter'])

#torch.tensor(epoch_mse)
#torch.save(epoch_mse,'EKF_epoch_mse')
#torch.save(EKF_MSE,'EKF_MSE')

stop_loss = torch.load('EKF_epoch_mse', map_location=device)
stop_loss = stop_loss.item()
EKF_MSE = torch.load('EKF_MSE', map_location=device)
print(stop_loss)
stop_loss = 0

0.07285962253808975


Calling the above training function:

In [12]:
import time

n_epochs = 250

stopping_list = [False, False, False, False, True]

UniRNN_hidden = torch.rand((1*UniRNN_num_layers, tMax, UniRNN_hidden_size),device=device)

BiRNN_hidden = torch.rand((2*BiRNN_num_layers, tMax, BiRNN_hidden_size),device=device)

UniLSTM_hidden = torch.rand((1*UniLSTM_num_layers, tMax, UniLSTM_hidden_size),device=device)
UniLSTM_cell = torch.zeros((1*UniLSTM_num_layers, tMax, UniLSTM_hidden_size),device=device)

BiLSTM_hidden = torch.rand((2*BiLSTM_num_layers, tMax, BiLSTM_hidden_size),device=device)
BiLSTM_cell = torch.zeros((2*BiLSTM_num_layers, tMax, BiLSTM_hidden_size),device=device)

UniRNN_training_loss_epoch = np.zeros(n_epochs)
BiRNN_training_loss_epoch = np.zeros(n_epochs)
UniLSTM_training_loss_epoch = np.zeros(n_epochs)
BiLSTM_training_loss_epoch = np.zeros(n_epochs)
Transformer_training_loss_epoch = np.zeros(n_epochs)

UniRNN_testing_loss_epoch = np.zeros(n_epochs)
BiRNN_testing_loss_epoch = np.zeros(n_epochs)
UniLSTM_testing_loss_epoch = np.zeros(n_epochs)
BiLSTM_testing_loss_epoch = np.zeros(n_epochs)
Transformer_testing_loss_epoch = np.zeros(n_epochs)

training_time = np.zeros(n_epochs)

i = 0
while i < n_epochs and not all(stopping_list):
    tic = time.perf_counter()
    for j, data in enumerate(trainloader):
        inputs, labels = data

        UniRNN_optimizer.zero_grad()
        BiRNN_optimizer.zero_grad()
        UniLSTM_optimizer.zero_grad()
        BiLSTM_optimizer.zero_grad()
        Transformer_optimizer.zero_grad()

        if stopping_list[0] == False:
            UniRNN_outputs, UniRNN_hidden = UniRNN(inputs, UniRNN_hidden.detach())
            UniRNN_train_loss = criterion(UniRNN_outputs, labels)
            UniRNN_train_loss.backward()
            UniRNN_optimizer.step()
            UniRNN_training_loss_epoch[i] += UniRNN_train_loss.item()
        if stopping_list[1] == False:
            BiRNN_outputs, BiRNN_hidden = BiRNN(inputs, BiRNN_hidden.detach())
            BiRNN_train_loss = criterion(BiRNN_outputs, labels)
            BiRNN_train_loss.backward()
            BiRNN_optimizer.step()
            BiRNN_training_loss_epoch[i] += BiRNN_train_loss.item()
        if stopping_list[2] == False:
            UniLSTM_outputs, (UniLSTM_hidden, UniLSTM_cell) = UniLSTM(inputs, UniLSTM_hidden.detach(), UniLSTM_cell.detach())
            UniLSTM_train_loss = criterion(UniLSTM_outputs, labels)
            UniLSTM_train_loss.backward()
            UniLSTM_optimizer.step()
            UniLSTM_training_loss_epoch[i] += UniLSTM_train_loss.item()
        if stopping_list[3] == False:
            BiLSTM_outputs, (BiLSTM_hidden, BiLSTM_cell) = BiLSTM(inputs, BiLSTM_hidden.detach(), BiLSTM_cell.detach())
            BiLSTM_train_loss = criterion(BiLSTM_outputs, labels)
            BiLSTM_train_loss.backward()
            BiLSTM_training_loss_epoch[i] += BiLSTM_train_loss.item()
            BiLSTM_optimizer.step()
        if stopping_list[4] == False:
            Transformer_outputs = Transformer(inputs)
            Transformer_train_loss = criterion(Transformer_outputs, labels)
            Transformer_train_loss.backward()
            Transformer_optimizer.step()
            Transformer_training_loss_epoch[i] += Transformer_train_loss.item()

        #UniRNN_scheduler.step()
        #BiRNN_scheduler.step()
        #UniLSTM_scheduler.step()
        #BiLSTM_scheduler.step()
        #Transformer_scheduler.step()

    UniRNN_training_loss_epoch[i] = UniRNN_training_loss_epoch[i] / (j+1)
    BiRNN_training_loss_epoch[i] = BiRNN_training_loss_epoch[i] / (j+1)
    UniLSTM_training_loss_epoch[i] = UniLSTM_training_loss_epoch[i] / (j+1)
    BiLSTM_training_loss_epoch[i] = BiLSTM_training_loss_epoch[i] / (j+1)
    Transformer_training_loss_epoch[i] = Transformer_training_loss_epoch[i] / (j+1)

    with (torch.no_grad()):
        for j, data in enumerate(testloader):            
            inputs, labels = data

            if stopping_list[0] == False:
                UniRNN_outputs, UniRNN_hidden = UniRNN(inputs, UniRNN_hidden.detach())
                UniRNN_test_loss = criterion(UniRNN_outputs, labels)
                UniRNN_testing_loss_epoch[i] += UniRNN_test_loss.item()
            if stopping_list[1] == False:
                BiRNN_outputs, BiRNN_hidden = BiRNN(inputs, BiRNN_hidden.detach())
                BiRNN_test_loss = criterion(BiRNN_outputs, labels)
                BiRNN_testing_loss_epoch[i] += BiRNN_test_loss.item()
            if stopping_list[2] == False:
                UniLSTM_outputs, (UniLSTM_hidden, UniLSTM_cell) = UniLSTM(inputs, UniLSTM_hidden.detach(), UniLSTM_cell.detach())
                UniLSTM_test_loss = criterion(UniLSTM_outputs, labels)
                UniLSTM_testing_loss_epoch[i] += UniLSTM_test_loss.item()
            if stopping_list[3] == False:
                BiLSTM_outputs, (BiLSTM_hidden, BiLSTM_cell) = BiLSTM(inputs, BiLSTM_hidden.detach(), BiLSTM_cell.detach())
                BiLSTM_test_loss = criterion(BiLSTM_outputs, labels)
                BiLSTM_testing_loss_epoch[i] += BiLSTM_test_loss.item()
            if stopping_list[4] == False:
                Transformer_outputs = Transformer(inputs)
                Transformer_test_loss = criterion(Transformer_outputs, labels)
                Transformer_testing_loss_epoch[i] += Transformer_test_loss.item()

        UniRNN_testing_loss_epoch[i] = UniRNN_testing_loss_epoch[i]  / (j+1)
        BiRNN_testing_loss_epoch[i] = BiRNN_testing_loss_epoch[i]  / (j+1)
        UniLSTM_testing_loss_epoch[i] = UniLSTM_testing_loss_epoch[i] / (j+1)
        BiLSTM_testing_loss_epoch[i] = BiLSTM_testing_loss_epoch[i] / (j+1)
        Transformer_testing_loss_epoch[i] = Transformer_testing_loss_epoch[i] / (j+1)

        if UniRNN_testing_loss_epoch[i] <= stop_loss:
            stopping_list[0] = True
        if BiRNN_testing_loss_epoch[i] <= stop_loss:
            stopping_list[1] = True
        if UniLSTM_testing_loss_epoch[i] <= stop_loss:
            stopping_list[2] = True
        if BiLSTM_testing_loss_epoch[i] <= stop_loss:
            stopping_list[3] = True
        if Transformer_testing_loss_epoch[i] <= stop_loss:
            stopping_list[4] = True
    
    toc = time.perf_counter()
    training_time[i] = toc-tic
    print("-----------------------------------------------------------------",
          "\n-----  Epoch:",i+1, "Completed in","{:3.0f}".format(training_time[i]),
          "s,","{:5.1f}".format((n_epochs-(i+1))/60*np.mean(training_time[0:i])), "Minutes Remaining -----")
    # num_epochs remaining = n_epochs - (i+1)
    print("-----  Deep Neural Networks  -----"," Train Loss : Test Loss  -----")
    print("-----  UniDirectional RNN:   -----",
          "{:10.3f}".format(UniRNN_training_loss_epoch[i]) +'  :'+"{:10.3f}".format(UniRNN_testing_loss_epoch[i])+'  -----')
    print("-----   BiDirectional RNN:   -----",
          "{:10.3f}".format(BiRNN_training_loss_epoch[i]) +'  :'+"{:10.3f}".format(BiRNN_testing_loss_epoch[i])+'  -----')
    print("-----  UniDirectional LSTM:  -----",
          "{:10.3f}".format(UniLSTM_training_loss_epoch[i]) +'  :'+"{:10.3f}".format(UniLSTM_testing_loss_epoch[i])+'  -----')
    print("-----  BiDirectional LSTM:   -----",
          "{:10.3f}".format(BiLSTM_training_loss_epoch[i]) +'  :'+"{:10.3f}".format(BiLSTM_testing_loss_epoch[i])+'  -----')
    print("-----      Transformer:      -----",
          "{:10.3f}".format(Transformer_training_loss_epoch[i]) +'  :'+"{:10.3f}".format(Transformer_testing_loss_epoch[i])+'  -----')
    i += 1

UniRNN_training_loss_epoch=np.trim_zeros(UniRNN_training_loss_epoch)
BiRNN_training_loss_epoch=np.trim_zeros(BiRNN_training_loss_epoch)
UniLSTM_training_loss_epoch=np.trim_zeros(UniLSTM_training_loss_epoch)
BiLSTM_training_loss_epoch=np.trim_zeros(BiLSTM_training_loss_epoch)
Transformer_training_loss_epoch=np.trim_zeros(Transformer_training_loss_epoch)

UniRNN_testing_loss_epoch =np.trim_zeros(UniRNN_testing_loss_epoch)
BiRNN_testing_loss_epoch=np.trim_zeros(BiRNN_testing_loss_epoch)
UniLSTM_testing_loss_epoch=np.trim_zeros(UniLSTM_testing_loss_epoch)
BiLSTM_testing_loss_epoch=np.trim_zeros(BiLSTM_testing_loss_epoch)
Transformer_testing_loss_epoch=np.trim_zeros(Transformer_testing_loss_epoch)

C:\Users\echo4\.conda\envs\pt\lib\site-packages\numpy\core\fromnumeric.py:3464: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\echo4\.conda\envs\pt\lib\site-packages\numpy\core\_methods.py:192: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


----------------------------------------------------------------- 
-----  Epoch: 1 Completed in  73 s,   nan Minutes Remaining -----
-----  Deep Neural Networks  -----  Train Loss : Test Loss  -----
-----  UniDirectional RNN:   -----   1056.814  :   922.002  -----
-----   BiDirectional RNN:   -----    943.298  :   762.167  -----
-----  UniDirectional LSTM:  -----   1112.332  :   979.215  -----
-----  BiDirectional LSTM:   -----   1039.670  :   781.559  -----
-----      Transformer:      -----      0.000  :     0.000  -----
----------------------------------------------------------------- 
-----  Epoch: 2 Completed in  73 s, 303.2 Minutes Remaining -----
-----  Deep Neural Networks  -----  Train Loss : Test Loss  -----
-----  UniDirectional RNN:   -----    857.508  :   796.132  -----
-----   BiDirectional RNN:   -----    637.136  :   486.946  -----
-----  UniDirectional LSTM:  -----    877.374  :   790.794  -----
-----  BiDirectional LSTM:   -----    617.387  :   502.019  -----
-----   

KeyboardInterrupt: 

Plotting

In [ ]:
class plotter():
    def __init__(self):
        a = 'Hello World'

    def avg_loss_vs_epoch(self, training_loss_list, legend_list):
        fig, Loss_combined_plot = plt.subplots()

        height = 4
        width = 6
        
        Loss_combined_plot.set_xlabel("Epoch")
        Loss_combined_plot.set_ylabel("Average Loss (RMSE)")
        Loss_combined_plot.set_title("Training and Testing Loss vs Time")
        Loss_combined_plot.figure.set_figwidth(width) # 6.4
        Loss_combined_plot.figure.set_figheight(height) # 6.4
        fig.tight_layout()

        t = range(len(training_loss_list[0]))
        i = 0
        for Loss in training_loss_list:
            Loss_combined_plot.plot(t, Loss, label = legend_list[i])
            i += 1
        Loss_combined_plot.legend(legend_list)
        
    def two_by_one_MSE_vs_time(self, MSE_list, legend_list):
        fig, MSE_combined_plot = plt.subplots(1,2)
    
        scale_factor = 1.5
        height = 6 * scale_factor
        width = 9 * scale_factor
        
        MSE_combined_plot[0].set_xlabel("Time (s)")
        MSE_combined_plot[0].set_ylabel("Root Mean Square Error (km)")
        MSE_combined_plot[0].set_title("Avg RMSE in Position vs Time")
        MSE_combined_plot[0].figure.set_figwidth(width) # 6.4
        MSE_combined_plot[0].figure.set_figheight(height) # 6.4
        MSE_combined_plot[1].set_xlabel("Time (s)")
        MSE_combined_plot[1].set_ylabel("Root Mean Square Error (km/s)")
        MSE_combined_plot[1].set_title("Avg RMSE in Velocity vs Time")
        MSE_combined_plot[1].figure.set_figwidth(width) # 6.4
        MSE_combined_plot[1].figure.set_figheight(height) # 6.4
        fig.tight_layout()

        t = range(len(time_span))
        i = 0
        for MSE in MSE_list:
            MSE_x = MSE.T[0].cpu().numpy()
            MSE_xv = MSE.T[1].cpu().numpy()
            MSE_y = MSE.T[2].cpu().numpy()
            MSE_yv = MSE.T[3].cpu().numpy()

            MSE_pos = np.sqrt( (MSE_x + MSE_y) / 2 )
            MSE_vel = np.sqrt( (MSE_xv + MSE_yv) / 2 )

            MSE_combined_plot[0].plot(t, MSE_pos, label = legend_list[i])
            MSE_combined_plot[1].plot(t, MSE_vel, label = legend_list[i])
            i += 1
        MSE_combined_plot[0].legend([legend_list])
        MSE_combined_plot[1].legend([legend_list])

    def two_by_two_MSE_vs_time(self, MSE_list, legend_list):
        fig, MSE_combined_plot = plt.subplots(2,2)
        scale_factor = 1.5
        height = 6 * scale_factor
        width = 9 * scale_factor
        
        MSE_combined_plot[0][0].set_xlabel("Time (s)")
        MSE_combined_plot[0][0].set_ylabel("Root Mean Square Error (km)")
        MSE_combined_plot[0][0].set_title("Avg RMSE in Horizontal Position vs Time")
        MSE_combined_plot[0][0].figure.set_figwidth(width) # 6.4
        MSE_combined_plot[0][0].figure.set_figheight(height) # 6.4
        
        MSE_combined_plot[1][0].set_xlabel("Time (s)")
        MSE_combined_plot[1][0].set_ylabel("Root Mean Square Error (km)")
        MSE_combined_plot[1][0].set_title("Avg RMSE in Vertical Position vs Time")
        MSE_combined_plot[1][0].figure.set_figwidth(width) # 6.4
        MSE_combined_plot[1][0].figure.set_figheight(height) # 6.4

        MSE_combined_plot[0][1].set_xlabel("Time (s)")
        MSE_combined_plot[0][1].set_ylabel("Root Mean Square Error (km/s)")
        MSE_combined_plot[0][1].set_title("Avg RMSE in Horizontal Velocity vs Time")
        MSE_combined_plot[0][1].figure.set_figwidth(width) # 6.4
        MSE_combined_plot[0][1].figure.set_figheight(height) # 6.4
        
        MSE_combined_plot[1][1].set_xlabel("Time (s)")
        MSE_combined_plot[1][1].set_ylabel("Root Mean Square Error (km/s)")
        MSE_combined_plot[1][1].set_title("Avg RMSE in Vertical Velocity vs Time")
        MSE_combined_plot[1][1].figure.set_figwidth(width) # 6.4
        MSE_combined_plot[1][1].figure.set_figheight(height) # 6.4
        fig.tight_layout()

        t = range(len(time_span))
        i = 0
        for MSE in MSE_list:
            MSE_x = MSE.T[0].cpu().numpy()
            MSE_xv = MSE.T[1].cpu().numpy()
            MSE_y = MSE.T[2].cpu().numpy()
            MSE_yv = MSE.T[3].cpu().numpy()

            MSE_combined_plot[0][0].plot(t, MSE_x, label = legend_list[i])
            #MSE_combined_plot[0][0].legend([legend_list[i]])
            
            MSE_combined_plot[1][0].plot(t, MSE_y, label = legend_list[i])
            #MSE_combined_plot[1][0].legend([legend_list[i]])
            
            MSE_combined_plot[0][1].plot(t, MSE_xv, label = legend_list[i])
            #MSE_combined_plot[0][1].legend([legend_list[i]])
            
            MSE_combined_plot[1][1].plot(t, MSE_yv, label = legend_list[i])
            #MSE_combined_plot[1][1].legend([legend_list[i]])
            i += 1

        MSE_combined_plot[0][0].legend(legend_list)
        MSE_combined_plot[1][0].legend(legend_list)
        MSE_combined_plot[0][1].legend(legend_list)
        MSE_combined_plot[1][1].legend(legend_list)


Training Plotting

In [ ]:
plotter = plotter()

#plotter.two_by_two_MSE_vs_time([EKF_MSE], ['Extended Kalman Filter'])

legend_list = ['Unidirectional RNN Training Loss', 'Unidirectional RNN Testing Loss']
plotter.avg_loss_vs_epoch([UniRNN_training_loss_epoch, UniRNN_testing_loss_epoch], legend_list)

legend_list = ['Bidirectional RNN Training Loss', 'Bidirectional RNN Testing Loss']
plotter.avg_loss_vs_epoch([BiRNN_training_loss_epoch, BiRNN_testing_loss_epoch], legend_list)

legend_list = ['Unidirectional LSTM Training Loss', 'Unidirectional LSTM Testing Loss']
plotter.avg_loss_vs_epoch([UniLSTM_training_loss_epoch, UniLSTM_testing_loss_epoch], legend_list)

legend_list = ['Bidirectional LSTM Training Loss', 'Bidirectional LSTM Testing Loss']
plotter.avg_loss_vs_epoch([BiLSTM_training_loss_epoch, BiLSTM_testing_loss_epoch], legend_list)

legend_list = ['Transformer Training Loss', 'Transformer Testing Loss']
plotter.avg_loss_vs_epoch([Transformer_training_loss_epoch, Transformer_testing_loss_epoch], legend_list)
'''
legend_list = ['Unidirectional RNN Testing Loss', 'Bidirectional RNN Testing Loss',
               'Unidirectional LSTM Testing Loss', 'Bidirectional LSTM Testing Loss']
plotter.avg_loss_vs_epoch([UniRNN_training_loss_epoch, 
                           BiRNN_testing_loss_epoch,
                           UniLSTM_training_loss_epoch,
                           BiLSTM_training_loss_epoch, legend_list)
'''
legend_list = ['Unidirectional RNN Testing Loss', 'Bidirectional RNN Testing Loss',
               'Unidirectional LSTM Testing Loss', 'Bidirectional LSTM Testing Loss',
               'Transformer Testing Loss']
plotter.avg_loss_vs_epoch([UniRNN_training_loss_epoch, 
                           BiRNN_testing_loss_epoch,
                           UniLSTM_training_loss_epoch,
                           BiLSTM_training_loss_epoch,
                           Transformer_training_loss_epoch], legend_list)


Trajectory MSE Plot

In [ ]:
trajectory_mse_func = nn.MSELoss(reduction='none')
tst = torch.load('testing_data', map_location = device)
trajectory_testloader = DataLoader(tst)

UniRNN_trajectory_MSE = torch.zeros(tMax, output_size, device=device)
BiRNN_trajectory_MSE = torch.zeros(tMax, output_size, device=device)
UniLSTM_trajectory_MSE = torch.zeros(tMax, output_size, device=device)
BiLSTM_trajectory_MSE = torch.zeros(tMax, output_size, device=device)
Transformer_trajectory_MSE = torch.zeros(tMax, output_size, device=device)

with (torch.no_grad()):
    for j, data in enumerate(trajectory_testloader):
        inputs, labels = data

        UniRNN_outputs, UniRNN_hidden = UniRNN(inputs, UniRNN_hidden.detach())
        BiRNN_outputs, BiRNN_hidden = BiRNN(inputs, BiRNN_hidden.detach())
        UniLSTM_outputs, (UniLSTM_hidden, UniLSTM_cell) = UniLSTM(inputs, UniLSTM_hidden.detach(), UniLSTM_cell.detach())
        BiLSTM_outputs, (BiLSTM_hidden, BiLSTM_cell) = BiLSTM(inputs, BiLSTM_hidden.detach(), BiLSTM_cell.detach())
        Transformer_outputs = Transformer(inputs)

        UniRNN_test_loss = criterion(UniRNN_outputs, labels)
        BiRNN_test_loss = criterion(BiRNN_outputs, labels)
        UniLSTM_test_loss = criterion(UniLSTM_outputs, labels)
        BiLSTM_test_loss = criterion(BiLSTM_outputs, labels)
        Transformer_test_loss = criterion(Transformer_outputs, labels)

        UniRNN_trajectory_MSE += trajectory_mse_func(UniRNN_outputs.squeeze(), labels.squeeze())
        BiRNN_trajectory_MSE += trajectory_mse_func(BiRNN_outputs.squeeze(), labels.squeeze())
        UniLSTM_trajectory_MSE += trajectory_mse_func(UniLSTM_outputs.squeeze(), labels.squeeze())
        BiLSTM_trajectory_MSE += trajectory_mse_func(BiLSTM_outputs.squeeze(), labels.squeeze())
        Transformer_trajectory_MSE += trajectory_mse_func(Transformer_outputs.squeeze(), labels.squeeze())

    UniRNN_testing_MSE = UniRNN_trajectory_MSE / (j+1)
    BiRNN_testing_MSE = BiRNN_trajectory_MSE / (j+1)
    UniLSTM_testing_MSE = UniLSTM_trajectory_MSE / (j+1)
    BiLSTM_testing_MSE = BiLSTM_trajectory_MSE / (j+1)
    Transformer_testing_MSE = Transformer_trajectory_MSE / (j+1)

This Plotter class contains all of the info to generate the plots used in the paper. Each method accepts it's own standardized input and handles any data preprocessing internally.

In [ ]:
mse_list = [UniRNN_testing_MSE, BiRNN_testing_MSE, UniLSTM_testing_MSE, BiLSTM_testing_MSE, Transformer_testing_MSE]
legend_list = ['Unidirectional RNN', 'Bidirectional RNN', 'Unidirectional LSTM', 'Bidirectional LSTM', 'Transformer']
plotter.two_by_one_MSE_vs_time(mse_list, legend_list)
plotter.two_by_two_MSE_vs_time(mse_list, legend_list)

###### 